# Notebook ETL (Extract Transform and Load) para prontuarios

### Visão Geral
Este é um pipeline ETL (Extract, Transform, Load) que processa prontuários médicos para extrair informações estruturadas sobre tratamento de pacientes com AR, avaliar eficácia terapêutica e preparar dados para análise.

In [1]:
from google.colab import files
from google.colab import drive
import pandas as pd

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Carregando os dados

In [3]:
path = "/content/drive/MyDrive/IMMUNED/ETL/"
ffile = "pacientes_AR_primeiro_2937.xlsx"

In [4]:
df = pd.read_excel(path+ffile)

In [5]:
df.head()

,Unnamed: 0,tipo,Numero Consulta,paciente,data_hora,descricao,especialidade,grade,idade,sexo
0,NaN,ANAMNESE,5041792,1000170,2024-10-01 10:52:00,# AMBULATÓRIO DE REUMATOLOGIA - ARTRITE REUMAT...,REUMATOLOGIA - ARTRITE REUMATOIDE,1460,73,F
1,NaN,EVOLUCAO,2286906,1000170,2023-05-10 13:28:00,# Artrite reumatoide\n \n\n PACIENTE PREENCHE ...,REUMATOLOGIA - ARTRITE REUMATOIDE,830,73,F
2,NaN,ANAMNESE,1560489,1000170,2022-01-12 08:51:00,# 70 ANOS \n \n\n # ARTRITE REUMATÓIDE ( DX HÁ...,REUMATOLOGIA - ARTRITE REUMATOIDE,1460,73,F
3,NaN,ANAMNESE,4763019,1000170,2025-02-12 13:37:00,# AMBULATÓRIO DE REUMATOLOGIA - ARTRITE REUMAT...,REUMATOLOGIA - ARTRITE REUMATOIDE,830,73,F
4,NaN,ANAMNESE,1103160,1000170,2022-06-15 14:09:00,70 ANOS - ambulatório de artrite reumatoide\n ...,REUMATOLOGIA - ARTRITE REUMATOIDE,830,73,F


In [6]:
df = df.drop_duplicates(subset=['descricao']).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2874 entries, 0 to 2873
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Unnamed: 0       0 non-null      float64       
 1   tipo             2874 non-null   object        
 2   Numero Consulta  2874 non-null   int64         
 3   paciente         2874 non-null   int64         
 4   data_hora        2874 non-null   datetime64[ns]
 5   descricao        2874 non-null   object        
 6   especialidade    2874 non-null   object        
 7   grade            2874 non-null   int64         
 8   idade            2874 non-null   int64         
 9   sexo             2874 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(4)
memory usage: 224.7+ KB


In [7]:
df.head()

,Unnamed: 0,tipo,Numero Consulta,paciente,data_hora,descricao,especialidade,grade,idade,sexo
0,NaN,ANAMNESE,5041792,1000170,2024-10-01 10:52:00,# AMBULATÓRIO DE REUMATOLOGIA - ARTRITE REUMAT...,REUMATOLOGIA - ARTRITE REUMATOIDE,1460,73,F
1,NaN,EVOLUCAO,2286906,1000170,2023-05-10 13:28:00,# Artrite reumatoide\n \n\n PACIENTE PREENCHE ...,REUMATOLOGIA - ARTRITE REUMATOIDE,830,73,F
2,NaN,ANAMNESE,1560489,1000170,2022-01-12 08:51:00,# 70 ANOS \n \n\n # ARTRITE REUMATÓIDE ( DX HÁ...,REUMATOLOGIA - ARTRITE REUMATOIDE,1460,73,F
3,NaN,ANAMNESE,4763019,1000170,2025-02-12 13:37:00,# AMBULATÓRIO DE REUMATOLOGIA - ARTRITE REUMAT...,REUMATOLOGIA - ARTRITE REUMATOIDE,830,73,F
4,NaN,ANAMNESE,1103160,1000170,2022-06-15 14:09:00,70 ANOS - ambulatório de artrite reumatoide\n ...,REUMATOLOGIA - ARTRITE REUMATOIDE,830,73,F


## Extraindo os dados

### Extração de Dados (Extract)
Função Principal: extract()
Esta função genérica processa texto não estruturado dos prontuários:

* Percorre cada linha da coluna 'descricao'
* Busca palavras-chave definidas em dicionários (marcadores, comorbidades, medicamentos).
* Quando encontra uma palavra-chave, captura o valor numérico seguinte.
* Suporta operadores de comparação (<, >, =)

1.   Marcadores Clínicos: extract_marcadores()
Extrai indicadores de atividade da doença:

* VHS: Velocidade de hemossedimentação
* Leucócitos: Contagem de glóbulos brancos
* PCR: Proteína C-reativa
* HAQ: Health Assessment Questionnaire (0-3, funcionalidade)
* DAS28: Disease Activity Score (avalia 28 articulações)
* CDAI: Clinical Disease Activity Index

2. Comorbidades: extract_comorbidades()
Identifica condições associadas:

* HAS (Hipertensão)
* DLP (Dislipidemia)
* FM (Fibromialgia)
* PRE_DP (Pré-diabetes)
* OP (Osteoporose)
* Hipotireoidismo

Cria flag binária comorbidade (0/1) indicando presença de qualquer comorbidade.

3. Medicamentos: extract_medicamentos()
Registra uso de DMARDs biológicos e JAK inibidores:

* JAK inibidores: Tofacitinibe, Upadacitinibe
* Anti-TNF: Adalimumabe, Etanercepte, Golimumabe, Infliximabe, Certolizumabe

In [8]:
# =============================================================================
# FUNÇÕES DE EXTRAÇÃO
# =============================================================================

def is_number(s):
    """Verifica se uma string contém números"""
    num = "0123456789"
    for i in num:
        if i in s:
            return True
    return False


def extract(df, mp: dict[str, list], decision):
    """Função genérica de extração"""
    for id, line in enumerate(df["descricao"]):
        curr_s, curr_w = "", ""
        search = False

        for word in line.split():
            word = word.lower()
            if word[-1] in ":\\<>;/'":
                word = word[:-1]

            if word in mp.keys():
                search = True
                curr_w = word
                continue

            if search:
                if word in "<>=":
                    curr_s = word
                    continue

                decision(df, id, word, curr_w, curr_s)
                curr_s = ""
                search = False

    return df


def extract_marcadores(df):
    """Extrai marcadores clínicos"""
    marcadores = {
        'vhs': [],
        'leuco': [],
        'pcr': [],
        'haq': [],
        'das28': [],
        'cdai': []
    }

    for k in marcadores.keys():
        df[k] = [None for i in range(len(df))]

    def decision(*args):
        df, id, word, curr_w, curr_s = args
        if is_number(word) and df.loc[id, curr_w] == None:
            df.loc[id, curr_w] = curr_s + word

    df = extract(df, marcadores, decision)
    return df


def extract_comorbidades(df):
    """Extrai comorbidades"""
    comorbidades = {
        'has': [],
        'dlp': [],
        'fm': [],
        'pre_dp': [],
        'op': [],
        'hipotireoidismo': []
    }

    df['comorbidade'] = [0 for i in range(len(df))]
    for k in comorbidades.keys():
        df[k] = [None for i in range(len(df))]

    def decision(*args):
        df, id, word, curr_w, curr_s = args
        if is_number(word):
            df.loc[id, curr_w] = curr_s + word
            df.loc[id, 'comorbidade'] = 1

    df = extract(df, comorbidades, decision)
    return df


def extract_medicamentos(df):
    """Extrai medicamentos"""
    medicamentos = {
        'tofacitinibe': [],
        'upadacitinibe': [],
        'adalimumabe': [],
        'etanercepte': [],
        'golimumabe': [],
        'infliximabe': [],
        'certolizumabe': []
    }

    for k in medicamentos.keys():
        df[k] = [0 for i in range(len(df))]

    def decision(*args):
        df, id, word, curr_w, curr_s = args
        if is_number(word):
            df.loc[id, curr_w] = 1

    df = extract(df, medicamentos, decision)
    return df

## Transformação e limpeza de dados

 ### Transformação (Transform)


1. Limpeza de Dados Numéricos
```
df[c] = (df[c].astype(str)
         .str.extract(r'(\d+[.,]?\d*)')
         .replace(',', '.', regex=True)
         .astype(float))
```
Converte strings com valores numéricos (ex: "vhs: 25", ">40") para float.

2. Filtragem de Pacientes Válidos
```
tipo_counts = df.groupby('paciente')['tipo'].nunique()
valid_ids = tipo_counts[tipo_counts == 2].index
```
Mantém apenas pacientes com ANAMNESE (consulta inicial) E EVOLUÇÃO (acompanhamento).

3.  Criação da Base Longitudinal
O código realiza um merge inteligente:

* Separa os registros:
  * an: Anamnese (baseline/t0) - pega o registro mais antigo
  * evo: Evolução (follow-up/t1) - pega o registro mais recente
* Renomeia colunas:
  * Marcadores t0: vhs_t0, das28_t0, etc.
  * Marcadores t1: vhs_t1, das28_t1, etc.

* Calcula tempo de tratamento:
```
merged['tempo_tratamento'] = (merged['data_hora_t1'] - merged['data_hora_t0']).dt.days
```

In [9]:
print("\n[ETAPA 1] Extraindo marcadores clínicos...")
df = extract_marcadores(df)

print("[ETAPA 2] Extraindo comorbidades...")
df = extract_comorbidades(df)

print("[ETAPA 3] Extraindo medicamentos...")
df = extract_medicamentos(df)

# =============================================================================
# TRANSFORMAÇÃO E LIMPEZA
# =============================================================================

print("\n[ETAPA 4] Limpando dados numéricos...")
cols = ['vhs', 'leuco', 'pcr', 'haq', 'das28', 'cdai',
        'comorbidade', 'has', 'dlp', 'fm', 'pre_dp', 'op', 'hipotireoidismo']

for c in cols:
    if c in df.columns:
        df[c] = (
            df[c]
            .astype(str)
            .str.extract(r'(\d+[.,]?\d*)')
            .replace(',', '.', regex=True)
            .astype(float)
        )

print(f"[DEBUG] 'sexo' preservado: {df['sexo'].unique()}")



[ETAPA 1] Extraindo marcadores clínicos...
[ETAPA 2] Extraindo comorbidades...
[ETAPA 3] Extraindo medicamentos...

[ETAPA 4] Limpando dados numéricos...
[DEBUG] 'sexo' preservado: ['F' 'M']


/tmp/ipython-input-975757326.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(',', '.', regex=True)


### Remoção de pacientes sem registro do tipo 'EVOLUCAO'

In [10]:
# =============================================================================
# REMOÇÃO DE PACIENTES SEM EVOLUÇÃO
# =============================================================================

print("\n[ETAPA 5] Filtrando pacientes com ANAMNESE e EVOLUÇÃO...")
tipo_counts = df.groupby('paciente')['tipo'].nunique()
valid_ids = tipo_counts[tipo_counts == 2].index

df = df[df['paciente'].isin(valid_ids)].reset_index(drop=True)
print(f"[INFO] Pacientes válidos: {len(valid_ids)}")


[ETAPA 5] Filtrando pacientes com ANAMNESE e EVOLUÇÃO...
[INFO] Pacientes válidos: 343


In [11]:

df_para_analise = df.copy()  # Salva estado para gráficos

### Juntando dados de evolução e anamnese

In [12]:
print("\n[ETAPA 6] Criando base longitudinal (t0 e t1)...")

evo = df[df['tipo'] == 'EVOLUCAO'].reset_index(drop=True)
an = df[df['tipo'] == 'ANAMNESE'].reset_index(drop=True)

# Pegar registros mais recentes
evo = evo.sort_values(by='data_hora', ascending=False)
evo = evo.drop_duplicates(subset=['paciente'], keep="first").reset_index(drop=True)

an = an.sort_values(by='data_hora', ascending=False)
an = an.drop_duplicates(subset=['paciente'], keep="last").reset_index(drop=True)

marker_cols = ['vhs', 'leuco', 'pcr', 'haq', 'das28', 'cdai', 'data_hora']

an.columns = [col + '_t0' if col in marker_cols else col for col in an.columns]
evo.columns = [col + '_t1' if col in marker_cols else col for col in evo.columns]

print("Colunas de 'an' após renomeação:", [c for c in an.columns if c in ['idade', 'sexo', 'vhs_t0', 'haq_t0']])
print("Colunas de 'evo' após renomeação:", [c for c in evo.columns if c in ['idade', 'sexo', 'vhs_t1', 'haq_t1']])

# Preparar colunas para merge
an_marker_cols = [col for col in an.columns if '_t0' in col]
evo_cols = evo.columns.tolist()

# ✅ MERGE: Agora 'idade' e 'sexo' existem!
merged = an[['paciente', 'idade', 'sexo'] + an_marker_cols].merge(
    evo,
    on='paciente'
).reset_index(drop=True)

# Calcular tempo de tratamento
merged['tempo_tratamento'] = (merged['data_hora_t1'] - merged['data_hora_t0']).dt.days

# Remover colunas duplicadas de idade/sexo do evo (se existirem)
if 'idade_y' in merged.columns:
    merged = merged.drop(columns=['idade_y', 'sexo_y'])
    merged = merged.rename(columns={'idade_x': 'idade', 'sexo_x': 'sexo'})

print(f"[INFO] Base longitudinal criada: {merged.shape}")
print(f"[INFO] Colunas finais incluem idade/sexo: {'idade' in merged.columns and 'sexo' in merged.columns}")



[ETAPA 6] Criando base longitudinal (t0 e t1)...
Colunas de 'an' após renomeação: ['idade', 'sexo', 'vhs_t0', 'haq_t0']
Colunas de 'evo' após renomeação: ['idade', 'sexo', 'vhs_t1', 'haq_t1']
[INFO] Base longitudinal criada: (343, 38)
[INFO] Colunas finais incluem idade/sexo: True


## Computando melhoras dos pacientes


4. Cálculo de Eficácia (Load)
Critérios de Melhora
```
improve_criteria = {
    'haq': lambda v0,v1: v1 <= v0-0.35,      # Redução ≥0.35 pontos
    'das28': lambda v0,v1: v1 <= v0*0.5,     # Redução ≥50%
}
```
* HAQ: Melhora clinicamente significativa = queda de 0.35+
* DAS28: Boa resposta EULAR = redução de 50%+

A coluna improvement recebe:
  * 1: Paciente melhorou (atende critério HAQ ou DAS28)
  * 0: Sem melhora significativa

Filtro de Tempo Mínimo
```
merged = merged[merged['tempo_tratamento'] >= 60]
```
Exclui tratamentos com menos de 2 meses (tempo insuficiente para avaliar resposta).

In [13]:
# =============================================================================
# COMPUTAR MELHORA DOS PACIENTES
# =============================================================================

print("\n[ETAPA 7] Calculando melhora dos pacientes...")

improve_criteria = {
    'haq': lambda v0, v1: v1 <= v0 - 0.35,
    'das28': lambda v0, v1: v1 <= v0 * 0.5,
}

merged["improvement"] = [None for _ in range(len(merged))]

for mark, criteria in improve_criteria.items():
    if f'{mark}_t0' in merged.columns and f'{mark}_t1' in merged.columns:
        for idx, (v0, v1) in enumerate(zip(merged[mark + '_t0'], merged[mark + '_t1'])):
            if merged.loc[idx, 'improvement'] is None and not pd.isna(v0) and not pd.isna(v1):
                merged.loc[idx, 'improvement'] = int(criteria(v0, v1))

merged['improvement'] = merged['improvement'].fillna(0).astype(int)

# =============================================================================
# FILTRO DE TEMPO MÍNIMO
# =============================================================================

print("\n[ETAPA 8] Removendo tratamentos < 60 dias...")
merged = merged[merged['tempo_tratamento'] >= 60].reset_index(drop=True)

print(f"[INFO] Base final: {merged.shape}")
print(f"\n{'='*60}")
print(f"Total de pacientes que melhoraram: {sum(merged['improvement'])}")
print(f"Porcentagem de melhora: {(sum(merged['improvement'])/len(merged)*100):.2f}%")
print(f"{'='*60}\n")

# =============================================================================
# SALVAR RESULTADO
# =============================================================================

output_file = path + 'prontuarios_ar_tratados.xlsx'
merged.to_excel(output_file, index=False)
print(f"[SUCESSO] Arquivo salvo: {output_file}")




[ETAPA 7] Calculando melhora dos pacientes...

[ETAPA 8] Removendo tratamentos < 60 dias...
[INFO] Base final: (323, 39)

Total de pacientes que melhoraram: 1
Porcentagem de melhora: 0.31%



/tmp/ipython-input-2536117743.py:20: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged['improvement'] = merged['improvement'].fillna(0).astype(int)


[SUCESSO] Arquivo salvo: /content/drive/MyDrive/IMMUNED/ETL/prontuarios_ar_tratados.xlsx


### Removendo pacientes que o tratamento durou menos de 2 meses (60 dias)

In [14]:
merged = merged[merged['tempo_tratamento'] >= 60].reset_index(drop=True)

In [15]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   paciente          323 non-null    int64         
 1   idade             323 non-null    int64         
 2   sexo              323 non-null    object        
 3   data_hora_t0      323 non-null    datetime64[ns]
 4   vhs_t0            135 non-null    float64       
 5   leuco_t0          227 non-null    float64       
 6   pcr_t0            231 non-null    float64       
 7   haq_t0            21 non-null     float64       
 8   das28_t0          8 non-null      float64       
 9   cdai_t0           32 non-null     float64       
 10  Unnamed: 0        0 non-null      float64       
 11  tipo              323 non-null    object        
 12  Numero Consulta   323 non-null    int64         
 13  data_hora_t1      323 non-null    datetime64[ns]
 14  descricao         323 non-

In [16]:
print(f"[*] Total de pacientes que melhoraram com o tratamento: {sum(merged['improvement']):>11}")
print(f"[*] Porcentagem de pacientes que melhoraram com o tratamento: {(sum(merged['improvement'])/len(merged)*100):>3.2f}%")

[*] Total de pacientes que melhoraram com o tratamento:           1
[*] Porcentagem de pacientes que melhoraram com o tratamento: 0.31%


---
# Analise de Dados


### Análise Exploratória
Gera visualizações com Plotly:

* Distribuição de idades
* Frequência por idade e sexo (empilhado)

In [17]:
print(f"Numero de pacientes:    {len(set(df['paciente']))}")
print(f"Numero de prontuarios: {len(df)}")

Numero de pacientes:    343
Numero de prontuarios: 2580


In [21]:
# =============================================================================
# ANÁLISE EXPLORATÓRIA
# =============================================================================
import plotly.graph_objects as go

print("\n[ANÁLISE] Gerando gráficos...")

# GRÁFICO 1: Frequência de Idades
print("\n1. Frequência de Idades")
freq = df_para_analise['idade'].value_counts().sort_index()

fig1 = go.Figure(data=[
    go.Bar(
        x=freq.index,
        y=freq.values,
        marker_color='blue'
    )
])

fig1.update_layout(
    title=dict(text='Frequência de Idades', font=dict(size=30)),
    xaxis_title='Idade',
    yaxis_title='Frequência',
    xaxis=dict(tickmode='linear')
)

fig1.show()
print(f"   - Faixa etária: {df_para_analise['idade'].min()} a {df_para_analise['idade'].max()} anos")

# GRÁFICO 2: Frequência por Idade e Sexo (CORRIGIDO)
print("\n2. Frequência por Idade e Sexo")

if 'sexo' in df_para_analise.columns and not df_para_analise['sexo'].isna().all():
    freq_sexo = df_para_analise.groupby(['idade', 'sexo']).size().unstack(fill_value=0).sort_index()

    sexos_disponiveis = freq_sexo.columns.tolist()
    print(f"   - Sexos encontrados: {sexos_disponiveis}")

    fig2 = go.Figure()

    if 'F' in sexos_disponiveis:
        fig2.add_trace(go.Bar(
            x=freq_sexo.index,
            y=freq_sexo['F'],
            name='Feminino',
            marker_color='pink'
        ))

    if 'M' in sexos_disponiveis:
        fig2.add_trace(go.Bar(
            x=freq_sexo.index,
            y=freq_sexo['M'],
            name='Masculino',
            marker_color='lightblue'
        ))

    fig2.update_layout(
        barmode='stack',
        title=dict(
            text='Frequência de Idades por Sexo (Barras Empilhadas)',
            font=dict(size=28)
        ),
        xaxis_title='Idade',
        yaxis_title='Frequência',
        xaxis=dict(tickmode='linear')
    )

    fig2.show()

    print(f"\n   - Distribuição por sexo:")
    print(df_para_analise['sexo'].value_counts())

    if 'F' in df_para_analise['sexo'].values and 'M' in df_para_analise['sexo'].values:
        print(f"\n   - Proporção F/M: {df_para_analise['sexo'].value_counts()['F'] / df_para_analise['sexo'].value_counts()['M']:.2f}:1")
else:
    print("   ⚠️ Coluna 'sexo' não disponível ou vazia")

# GRÁFICO 3: Distribuição de Melhora
print("\n3. Distribuição de Melhora no Tratamento")

fig3 = go.Figure(data=[
    go.Pie(
        labels=['Sem Melhora', 'Com Melhora'],
        values=[
            len(merged[merged['improvement'] == 0]),
            len(merged[merged['improvement'] == 1])
        ],
        marker=dict(colors=['#ff9999', '#66b3ff']),
        textinfo='label+percent+value'
    )
])

fig3.update_layout(
    title=dict(
        text='Resposta ao Tratamento (≥60 dias)',
        font=dict(size=26)
    )
)

fig3.show()

print("\n[CONCLUÍDO] ETL finalizado com sucesso! ✅")


[ANÁLISE] Gerando gráficos...

1. Frequência de Idades


   - Faixa etária: 31 a 95 anos

2. Frequência por Idade e Sexo
   - Sexos encontrados: ['F', 'M']



   - Distribuição por sexo:
sexo
F    2278
M     302
Name: count, dtype: int64

   - Proporção F/M: 7.54:1

3. Distribuição de Melhora no Tratamento



[CONCLUÍDO] ETL finalizado com sucesso! ✅


#### Pontos de Atenção
* Limitações
  * Extração por regex simples: Pode falhar em textos complexos
  * Última anamnese vs primeira: O código pega a última anamnese (keep="last"), mas comentário diz "mais recente"
  * Critérios de melhora: Não considera CDAI, VHS ou PCR individualmente
  * Dados faltantes: Pacientes sem HAQ ou DAS28 em ambos tempos ficam com improvement=0

* Forças
  * Estrutura modular e reutilizável
  * Validação de pacientes com seguimento completo
  * Critérios clínicos estabelecidos (EULAR)
  * Preserva rastreabilidade (IDs, datas)

#### Resultado Final
A base merged contém:

#### Pacientes únicos com anamnese + evolução
  * Dados longitudinais (t0 e t1)
  * Informações clínicas: marcadores, comorbidades, medicamentos
  * Desfecho primário: variável improvement (0/1)
  * Tempo de tratamento calculado
